In [3]:
import sys
sys.path.append("../src")

import pandas as pd

import plotly.graph_objs as go
import plotly.offline as py

import utilities as u
import constants as c

In [95]:
dfg = pd.read_excel("../data/data.xlsx", "liquid_m", index_col=0)

In [96]:
dfg

,Total,CC - Caixa enginyers,Diposit - Caixa enginyers,C. sin nómina - ING,C. nómina - ING,C. Naranja - ING,Diposit - ING,DeGiro líquid,Kraken líquid,Bunq,Revolut
Date,,,,,,,,,,,
2011-12-01,NaN,43.42,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01,NaN,89.23,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-02-01,NaN,353.79,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-01,NaN,553.06,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-04-01,NaN,534.36,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-01,NaN,875.41,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-06-01,NaN,939.98,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-07-01,NaN,637.97,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-08-01,NaN,346.19,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
